In [1]:
# This is code derived from online
# It currently works with no issue
# It must be run while the venv is active from the CLI

### SOURCE ###
# https://www.etedal.net/2020/04/pybullet-panda.html

import pybullet as p
import pybullet_data
import numpy as np
import h5py
import time
import os
import math

#DONT FORGET TO ACTIVATE VIRTUAL ENVIORNMENT: source pyvenv/bin/activate

pybullet build time: Nov 28 2023 23:51:11


### This is a function I defined to get the joint states at a given time

In [2]:
# Get current joint positions
def current_joint_positions(robot_id, joint_indices):
    current_joint_positions = []
    for joint_index in joint_indices:
        joint_state = p.getJointState(robot_id, joint_index)
        current_joint_positions.append(joint_state[0])  # Joint position is the first element of the tuple returned by getJointState
    
    return current_joint_positions

### Simloop

This loop runs for a specified number of simulations and spawns in a block object at a random position on the table and has the arm move towards the block's postition using inverse kinematics. The robot then grasps the object before stopping the simulation and saving the joint states of the robot during the simulation and saves them to a list of joint states from every simulation. I can make this a pandas dataframe too if it is more convinient. 

In [3]:
# Change this variable to change how many simulations are run
num_sims = 10

# Joint states from each simulation is published here
sim_results = []


for loop in range(num_sims):
    runsim = True

    p.connect(p.GUI)
    p.setGravity(0,0,-9.81)


    urdfRootPath=pybullet_data.getDataPath()
    kinovaUid = p.loadURDF('./j2s6s300.urdf',useFixedBase=True)

    # Dictionary of Robot Active Joints and Home Position
    arm_Joints = [2, 3, 4, 5, 6, 7]
    finger_joints = [9, 10, 11, 12, 13, 14]

    #homePos = [0.0, 2.9, 1.3, 4.2, 1.4, 0.0, 0, 0, 0]
    homePos = [math.pi, math.pi, math.pi, math.pi, math.pi, math.pi]

    homeStateDict = dict(zip(arm_Joints, homePos))

    # Sets robot home position before loading other objects
    for joint, pos in homeStateDict.items():
        p.resetJointState(kinovaUid, joint, pos)

    tableUid = p.loadURDF(os.path.join(urdfRootPath, "table/table.urdf"),basePosition=[0.5,0,-0.65])


    # Create a block collision shape
    blockShape = p.createCollisionShape(p.GEOM_BOX, halfExtents=[0.03, 0.03, 0.03])

    # Create a block rigid body
    blockBody = p.createMultiBody(baseMass=1.0, baseCollisionShapeIndex=blockShape)

    # Set the initial position and orientation of the block
    #initialPosition = [0.7, 0, 0.1]
    x_min, x_max = -0.2, 1.0  # Example limits for x coordinate
    y_min, y_max = -0.5, 0.5  # Example limits for y coordinate

    # Generate random x and y coordinates within the defined limits
    initial_x = np.random.uniform(x_min, x_max)
    initial_y = np.random.uniform(y_min, y_max)

    
    
    initialPosition = [initial_x, initial_y, 0.1]

    initialOrientation = p.getQuaternionFromEuler([0, 0, 0])  # no rotation
    p.resetBasePositionAndOrientation(blockBody, initialPosition, initialOrientation)

    objectUid = blockBody

    p.resetDebugVisualizerCamera(cameraDistance=1.5, cameraYaw=0, cameraPitch=-40, cameraTargetPosition=[0.55,-0.35,0.2])


    js_list = []


    state_durations = [0.25,0.5,0.25]
    control_dt = 1./240.
    p.setTimestep = control_dt
    state_t = 0.
    current_state = 0

    MAX_FINGER_POS = 1.5

    ikSolver = 0


    while runsim == True:
        p.stepSimulation()    
        
        js_list.append(current_joint_positions(kinovaUid, arm_Joints))

        state_t += control_dt
        p.configureDebugVisualizer(p.COV_ENABLE_SINGLE_STEP_RENDERING) 
        
        if current_state == 0:
            # Sets robot home position before loading other objects
            for joint, pos in homeStateDict.items():
                p.setJointMotorControl2(kinovaUid, joint, p.POSITION_CONTROL, pos)


            
        if current_state == 1:
            # Moves arm towards object, I can change the orientation at which the arm grabs the object if needed
            objPos = p.getBasePositionAndOrientation(objectUid)[0]


            jointPoses = p.calculateInverseKinematics(kinovaUid,
                                                            8,
                                                            objPos,
                                                            solver=ikSolver)

                
                
            for i in range(6):     
                
                p.setJointMotorControl2(bodyIndex=kinovaUid,
                                        jointIndex=i+2,
                                        controlMode=p.POSITION_CONTROL,
                                        targetPosition=jointPoses[i],
                                        targetVelocity=0,
                                        #force=500,
                                        positionGain=0.03,
                                        velocityGain=1
                                                        )
                                    

        if current_state == 2:
            # Closes the fingers to grab the object

            for i in finger_joints:
                if i % 2 == 1:
                    p.setJointMotorControl2(kinovaUid, i,
                                        p.POSITION_CONTROL, 0.6 * MAX_FINGER_POS, force = 200)
                    p.setJointMotorControl2(kinovaUid, i + 1,
                                        p.POSITION_CONTROL, 0.5 * MAX_FINGER_POS, force = 200)

            
        

        if state_t >state_durations[current_state]:
            current_state += 1
            if current_state >= len(state_durations):
                sim_results.append(js_list) # Appends the joint states of arm during simulation at the sampling frequency control_dt
                p.disconnect()
                runsim = False
            state_t = 0

            

        


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Intel
GL_RENDERER=Mesa Intel(R) UHD Graphics (CML GT2)
GL_VERSION=4.6 (Core Profile) Mesa 21.2.6
GL_SHADING_LANGUAGE_VERSION=4.60
pthread_getconcurrency()=0
Version = 4.6 (Core Profile) Mesa 21.2.6
Vendor = Intel
Renderer = Mesa Intel(R) UHD Graphics (CML GT2)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf:

In [4]:
print(sim_results)

[[[3.1415926523319895, 3.14159265153768, 3.141592657216762, 3.141592666448117, 3.141592653571871, 3.141592653589793], [3.1415926498913076, 3.1415926487315198, 3.1415926637824074, 3.1415926787529878, 3.141592653555687, 3.141592653589793], [3.141592646779343, 3.1415926455535397, 3.1415926719351095, 3.141592690150661, 3.141592653541342, 3.141592653589793], [3.141592643365824, 3.141592642257204, 3.1415926807714922, 3.14159270061955, 3.141592653528537, 3.141592653589793], [3.141592644381761, 3.141592643388455, 3.141592698724839, 3.141592692658397, 3.141592650613281, 3.14159266640128], [3.1415926452963494, 3.1415926444087625, 3.1415927156539705, 3.1415926851842095, 3.141592647944911, 3.14159267804975], [3.141592646119695, 3.141592645328274, 3.141592731324391, 3.141592678293059, 3.14159264551542, 3.1415926886170817], [3.1415926468608966, 3.1415926461565413, 3.141592745637201, 3.1415926720116905, 3.141592643314306, 3.141592698166078], [3.1415926475281464, 3.1415926469023394, 3.1415927585816883

### Gets the joint info for all joints in specified robot model

In [5]:
# Get the number of joints in the robot
numJoints = p.getNumJoints(kinovaUid)

# Iterate over each joint to get its information
for jointIndex in range(numJoints):
    jointInfo = p.getJointInfo(kinovaUid, jointIndex)
    jointName = jointInfo[1].decode("utf-8")  # Decode the joint name
    print("Joint Index:", jointIndex, "Joint Name:", jointName)


error: Not connected to physics server.